In [10]:
from InitialGraphs import Graphdata
from InitialPattern import Pattern
from GraphPattern import PatternMarge
import NPGSAlgorithm as na
import time
import os
import math

fileName = 'soc-political-retweet'
pers = 20 # N-hop Pattern Filtering Ratio
psize = 100 # 𝐹𝑟𝑒𝑞𝑢𝑒𝑛𝑡 𝑃𝑎𝑡𝑡𝑒𝑟𝑛 𝑀𝑎𝑛𝑎𝑔𝑚𝑒𝑛𝑡 Table Size
batchsize = 1
windowsize = 1
DxMtrix = {}
ExceptionList = {}
iterations = 0
iterations2 = 0
FPMT = {}
th2 = batchsize * windowsize
avg_time = 0

cnDict = {}

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = (len(dict)-1)*n//100
    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

IGPath = './dataset/' + fileName + '/' + fileName + '.edges' #sys.argv[1]
lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels' #sys.argv[2]

def neighborScore(v, u):
    degMax = DataT0.IG.vertex(v)[1]
    degSame = len((DataT0.IG.vertex(v)[2].keys() & DataT0.IG.vertex(u)[2].keys()))
    degRatio = DataT0.IG.vertex(u)[1]/DataT0.IG.vertex(v)[1]
            
    score = (degSame/degMax) + degRatio
    
    return score

def OPatternMarge(v):
    score = {}
    NeighborNode = DataT0.IG.vertex(v)[2]
    
    for u in NeighborNode:
        if u in cnDict:
            continue
        
        nbRe = NeighborNode[u].split('/')
        nbRe.reverse()
        PatternRevers = '/'.join(nbRe)
        if NeighborNode[u] in DataP0.NhP.keys() or PatternRevers in DataP0.NhP.keys():
            score[u] = neighborScore(v, u)
    
    if score:
        su = list(sorted(score.items(), key = lambda item: item[1], reverse = True))[0]
        DataT0.IG.del_edge(v, su[0])
        MargeNode = v + '/' + su[0]
        MargePattern = DataT0.lbDict[v] + '/' + DataT0.lbDict[su[0]]
        return MargeNode, MargePattern
    else:
        return 0, 0
    
def NPatternMarge(MargeNode, MargePattern):
        MargeNodeList = MargeNode.split('/')
        frontNode = MargeNodeList[0]
        backNode = MargeNodeList[-1]
        frontMax = [0,0]
        backMax = [0,0]
        UnionPattern1 = {}
        UnionPattern2 = {}
        frontScore = {}
        backScore = {}

        for front in DataT0.IG.vertex(frontNode)[2].keys():
            if front in cnDict or front in MargeNodeList:
                continue
            UnionPattern1[front] = DataT0.lbDict[front] + '/' + MargePattern
            
        for back in DataT0.IG.vertex(backNode)[2].keys():
            if back in cnDict or back in MargeNodeList:
                continue
            UnionPattern2[back] = MargePattern + '/' + DataT0.lbDict[back]
                
        for union in UnionPattern1.keys():
            ptlist = UnionPattern1[union].split('/')
            ptlist.reverse()
            PatternRevers = '/'.join(ptlist)
            if UnionPattern1[union] in DataP0.NhP.keys() or PatternRevers in DataP0.NhP.keys():
                frontScore[union] = neighborScore(frontNode, union)
        
        for union in UnionPattern2.keys():
            ptlist = UnionPattern2[union].split('/')
            ptlist.reverse()
            PatternRevers = '/'.join(ptlist)
            if UnionPattern2[union] in DataP0.NhP.keys() or PatternRevers in DataP0.NhP.keys():
                backScore[union] = neighborScore(backNode, union)
        
        if frontScore:
            frontMax = list(sorted(frontScore.items(), key = lambda item: item[1], reverse = True))[0]

        if backScore:
            backMax = list(sorted(backScore.items(), key = lambda item: item[1], reverse = True))[0]
            
        if frontMax[1] >= backMax[1] and frontMax[1] != 0:
            DataT0.IG.del_edge(frontNode, frontMax[0])
            MargeNode = frontMax[0] + '/' + MargeNode
            MargePattern = UnionPattern1[frontMax[0]]
            
        elif frontMax[1] < backMax[1] and backMax[1] != 0:
            DataT0.IG.del_edge(backNode, backMax[0])
            MargeNode = MargeNode + '/' + backMax[0]
            MargePattern = UnionPattern2[backMax[0]]
            
        else:
            return MargeNode, MargePattern, 0
            
        return MargeNode, MargePattern, 1
    
Separator = ','

DataT0 = Graphdata(IGPath, lbPath, Separator)

DataP0 = Pattern(DataT0.OhP, ExceptionList, pers, psize)
if '3524' in DataT0.nList:
    print('check')
PM = PatternMarge(DataT0.IG, DataP0.NhP, DataT0.lbDict, DataT0.nList)
print('check2')


while(True):
    print('check3')
    eList = [] # edge list
    non = len(DataT0.nList) # number of nodes
    print(non)
    print('check4')
    if non <= 0:
        break
    
    node = DataT0.nList[0]
    print(node)
    mn, mp = OPatternMarge(node) #marge node, marge pattern, one-hop pattern marge

    if mn == 0:
        DataT0.nList.remove(node)
        continue

    b = 1 #boolean
    while (b > 0):
        mn, mp, b = NPatternMarge(mn, mp) #N-hop pattern marge
        

check
check2
check3
0
check4


In [110]:
class PatternMarge:
    def __init__(IG, NhP, lbDict, nList):
        IG = IG # Initial Graph
        NhP = NhP # N-hop pattern
        lbDict = lbDict # label dict
        cnDict = {} # changed node dict
        nList = nList # node lisst
        pidDict = {} # pattern id dict {Marge pattern : Pattern ID}
        pCont = {}
        
        pDict = {} # pattern dict {Marge pattern : Marge node}
        eDict = {} # edge dict {Marge Node : Edge List}
        nopDict = {} # number of patterns dict

        while(True):
            eList = [] # edge list
            non = len(nList) # number of nodes

            if non <= 0:
                break
            
            node = nList[0]
            mn, mp = OPatternMarge(node) #marge node, marge pattern, one-hop pattern marge

            if mn == 0:
                nList.remove(node)
                continue

            b = 1 #boolean
            while (b > 0):
                mn, mp, b = NPatternMarge(mn, mp) #N-hop pattern marge
            
            mpRe = mp.split('/')
            mpRe.reverse()
            mpReverse = '/'.join(mpRe)
            
            if mpReverse in NhP.keys():
                mnRe = mn.split('/')
                mnRe.reverse()
                mn = '/'.join(mnRe)
                mp = mpReverse
            
            dList = mn.split('/') #delete list
            for dValue in dList:
                nList.remove(dValue)
                degList = list(IG.vertex(dValue)[2].keys()) ##degree List
                eList.extend(degList)
                cnDict[dValue] = mn
                
            eDict[mn] = eList
            
            if mp not in pDict:
                pDict[mp] = [mn]
                nopDict[mp] = len(eList)
            else:
                pDict[mp].append(mn)
                nopDict[mp] = nopDict[mp] + len(eList)
        
        pList = list(sorted(nopDict.items(), key = lambda item: item[1], reverse = True)) #pattern list
        prList = [] #pattern rank list
        
        for pl in pList:
            prList.append(pl[0])
        
        c0 = 0
        for prValue in prList:
            P0 = "P" + str(c0)
            pidDict[prValue] = P0
            c0 += 1
        
        mn_pidnDict = {} # PatternID-Number {Marge Node : PatternID-Number}
        pidn_pidDict = {} # {PatternID-Number : PatternID}
        pidn_mnDict = {} # {PatternID-Number : Marge Node}
        
        for mp in pDict:
            PID = pidDict[mp] # Pattern ID
            c1 = 0
            for mn in pDict[mp]:
                PIDN = PID + '-' + str(c1) # Pattern ID Number
                mn_pidnDict[mn] = PIDN
                pidn_mnDict[PIDN] = mn
                pidn_pidDict[PIDN] = PID
                c1 += 1
                
            pCont[mp] = c1 #Pattern ID Count
        
        GSe = {} # summarized graph edge {SGn:SGlabel}

        for nValue in eDict:
            dList = []
            for eValue in eDict[nValue]:
                if eValue in cnDict:
                    dList.append(eValue)
                    eDict[nValue].append(cnDict[eValue])
                    
            for dValue in dList:
                eDict[nValue].remove(dValue)
            eDict[nValue] = list(set(eDict[nValue]))

            if nValue in mn_pidnDict:
                SGn = mn_pidnDict[nValue] # summarized graph node
            else:
                SGn = nValue

            for eValue in eDict[nValue]:
                if eValue in mn_pidnDict:
                    SGnn = mn_pidnDict[eValue] #summarized graph neigbor node
                else:
                    SGnn = eValue
                    
                if SGn not in GSe:
                    GSe[SGn] = [SGnn]
                else:
                    GSe[SGn].append(SGnn)
                    
        SG = [] # output graph OR summarized graph
        SGnl = {} # summarized graph node label {nodes : label}
        for SGn in GSe:
            if SGn not in SGnl:
                if SGn in lbDict:
                    SGnl[SGn] = lbDict[SGn]
                elif SGn in pidn_pidDict:
                    SGnl[SGn] = pidn_pidDict[SGn]

            for SGnn in GSe[SGn]:
                if SGn == SGnn:
                    continue
                if [SGn, SGnn] not in SG and [SGnn, SGn] not in SG:
                    SG.append([SGn, SGnn])
                if SGnn not in SGnl:
                    if SGnn in lbDict:
                        SGnl[SGnn] = lbDict[SGnn]
                    elif SGnn in pidn_pidDict:
                        SGnl[SGnn] = pidn_pidDict[SGnn]
                        
                        
  


        
    
            


15322


In [91]:
fileName = 'PROTEINS-full'
IGPath = './dataset/' + fileName + '/' + fileName + '.edges' #sys.argv[1]
lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels' #sys.argv[2]


def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

input_file = [IGPath, lbPath]
file_size1 = 0
for ip in input_file:
    file_size1 = file_size1 + os.path.getsize(ip)
    print(ip.split('.')[2], convert_size(os.path.getsize(ip)))
print('InPut File Size:', convert_size(file_size1), 'bytes')
e1 = time.time() - s1
print(f'Th : {Th1}, FPMTs: {FPMTs}, NLs: {NLs}, Es: {Es}, Time: {e1}')

edges 3.13 MB
node_labels 256.94 KB
InPut File Size: 3.39 MB bytes
Th : 31786, FPMTs: 100, NLs: 108, Es: 128, Time: 44.75308656692505
